<h1><center>Laboratorio 3: Clasificación y Clustering 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Diego Irarrázaval

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Jonathan Espínola
- Nombre de alumno 2: Tomas Valencia


### **Link de repositorio de GitHub:** `https://github.com/Johnny952/labDataScience`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Manejo de DataFrames con la libreria `pandas`.
- Exploración del dataset y los resultados a través de visualizaciones interactivas usando `plotly`.
- Clasificación en `scikit-learn`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 4/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Aplicar y aprovechar las ventajas que nos ofrece la libreria `pandas`.
- Utilizar `plotly` para obtener información gráfica del dataset.
- Aplicar las ventajas que nos ofrece crear un pipeline.
- Visualizar el funcionamiento de clasificadores.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

Requirement already up-to-date: plotly in c:\users\jonathan\anaconda3\lib\site-packages (4.14.3)
  Created wheel for umap-learn: filename=umap_learn-0.5.1-py3-none-any.whl size=76570 sha256=8ce039982bfdfdd7138a5a45e07259316d63de220db95814573fe9a56bdfa468
  Stored in directory: c:\users\jonathan\appdata\local\pip\cache\wheels\95\85\b7\b4b7040e49367b6d1505d7e8fb57e3e79b22fa6ac26f72520b
  Created wheel for pynndescent: filename=pynndescent-0.5.2-py3-none-any.whl size=51353 sha256=8f2a933e0cb44c23cebdf1d9fc8cc22d220855a6db7eb905ae4b1f31538880a9
  Stored in directory: c:\users\jonathan\appdata\local\pip\cache\wheels\bb\81\5d\c78c8f15f3c815197129f91ec435d8ae0bb0a7a856d962c3ce
Successfully built umap-learn pynndescent


# 1. Predicciones futboleras.

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

Para este laboratorio se deberá trabajar con los csv ```FullData.csv```, donde deberán aplicar algoritmos de Clustering y clasificación información relevante de jugadores de futbol.

Para comenzar cargue el dataset señalado y visualice a traves de un head los atributos que posee el dataset.

In [141]:
# Usar solamente si utilizan Collab.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    dir = '//content//drive//My Drive//CuerpoDocente//Laboratorios//Lab3//'
except ModuleNotFoundError:
    dir = ''

In [142]:
df_players = pd.read_csv(dir+'stats_players.csv')
df_players.head()

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185 cm,80 kg,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170 cm,72 kg,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174 cm,68 kg,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182 cm,85 kg,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193 cm,92 kg,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11


De la tabla se reconoce:

1. `Name`: Nombre del jugador
2. `Nationality`: Nacionalidad del jugador
3. `National_Position`: Posición en la selección, este valor solo existe cuando el jugador es seleccionado nacional.
4. `Club_Position`: Posición del jugador. Este parámetro tiene valor en gran parte de los jugadores, existen casos como `Res` o `Sub` que no los deberá considerar en la respectiva sección.
5. `Height`: Altura del jugador en centímetros. Si nota esta variable tiene un string, transforme a int.
6. `Weight`: Peso del jugador en kilogramos. Si nota esta variable tiene un string, transforme a int.
7. `Preffered_Foot`: Pie con que prefiere patear.
8. `Age`: Edad
9. `Work_Rate`: Frecuencia con que trabaja.

Mientras que el resto de las características representan estadísticas personales (de acuerdo a la habilidad) para cada uno de los jugadores.


## 1.1 Predicción de seleccionados nacionales. (2 puntos)

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


### 1.1.1 Generación de Labels para la Clasificación

Tras el trágico despido de la mítica mascota de la población de Maipú, *Renacín*, decide adentrarse en la predicción futbolera. Para esto, utilizando los datos cargados en las celdas anteriores, le consulta si es posible predecir **si un jugador de football será seleccionado nacional en base a sus estadísticas**

En este primer experimento de clasificación comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. Hecho esto, ¿Como se distribuyen los datos para cada clase?, comente lo que observa.


In [143]:
### Código aquí ###
df_players.loc[df_players['National_Position'].isna(), 'National_Position'] = 0
df_players.loc[df_players['National_Position'] != 0, 'National_Position'] = 1
df_players['National_Position'] = df_players['National_Position'].astype(np.uint8)

In [144]:
print(f"Cantidad de datos etiquetados como seleccionados (1): {(df_players.National_Position == 1).sum()}")
print(f"Cantidad de datos etiquetados como no-seleccionados (0): {(df_players.National_Position == 0).sum()}")

Cantidad de datos etiquetados como seleccionados (1): 1075
Cantidad de datos etiquetados como no-seleccionados (0): 16513


### 1.1.2 Camino a la Clasificación

<p align="center">
  <img src="http://vignette3.wikia.nocookie.net/inciclopedia/images/c/cd/Patada_Medel_Messi.gif/revision/latest?cb=20150705215005" width="300">
</p>



Si bien los atributos de interés se encuentran en su mayoría limpios, existen valores mal ingresados al `DataFrame`. Para esto: 

1. Transforme a enteros todos los valores numéricos de la tabla que considere pertinentes. Por ejemplo: El peso aparece como strings y además estos continene la unidad. En este caso, debería transformar estos valores a enteros. Puede usar directamente funciones sobre el DataFrame (como `apply`) o el submódulo de las Series `str`.
2. Luego, aplique un ```ColumnTransformer``` para pre-procesar los datos del DataFrame de la siguiente forma:

Aplicar ```OneHotEncoder()``` sobre los atributos:

```python
['Preffered_Foot', 'Work_Rate']
```

Aplicar ```MinMaxScaler()``` sobre los atributos:

```python
['Age']
```

Aplicar ```StandardScaler()``` sobre los atributos:

```python
['Height', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 'Reactions','Interceptions', 'Vision', 'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys']
```

**Nota 2**: Puede copiar/pegar directamente las columnas en el `ColumnTransformer`.


Hecho lo anterior, defina al menos 3 pipelines para la clasificación donde anexe las transformaciones señaladas variando entre cada pipeline solo los clasificadores (puede escoger cualquiera de los clasificadores vistos en clases).

**To-Do:**

- [ ] Modificar valores mal ingresados.
- [ ] Aplicar ```ColumnTransformer``` sobre los datos a través de un pipeline.
- [ ] Justificar porque se utilizan estos pre-procesamientos.
- [ ] Crear 3 pipelines con diferentes clasificadores.
- [ ] No entrenar los datos aún.

**Nota:** No es necesario entrenar los clasificadores aún.

In [145]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

### Código aquí ###
df_players.Height = df_players.Height.str.replace(' cm', '').astype(int)
df_players.Weight = df_players.Weight.str.replace(' kg', '').astype(int)

ct = ColumnTransformer(
    [
        ("onehot", OneHotEncoder(), ['Preffered_Foot', 'Work_Rate']),
        ("minmax", MinMaxScaler(), ['Age']),
        ("stdscaler", StandardScaler(), ['Height', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 'Reactions','Interceptions', 'Vision', 'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys'])
    ]
)

pipeline1 = Pipeline(
    [
        ("ct", ct),
        ("tree", DecisionTreeClassifier())
    ]
)

pipeline2 = Pipeline(
    [
        ("ct", ct),
        ("svm", RandomForestClassifier())
    ]
)

pipeline3 = Pipeline(
    [
        ("ct", ct),
        ("lr", KNeighborsClassifier(n_neighbors=7))
    ]
)

### 1.1.3 Métricas de evaluación + Opcional

<p align="center">
  <img src="https://media4.giphy.com/media/3orif8r5vpbAdQU3iU/200w.gif?cid=82a1493btc1h43n1h4c59p13kjv6adugso4lo4292zbgt66b&rid=200w.gif&ct=g" width="300">
</p>

Interesado por saber que tan bien funciona su clasificador, Renacin le solicita que construya **a mano** las siguientes métricas de evaluación para el caso de **clasificación binaria**:

- Accuracy (Exactitud):

$$Accuracy = \dfrac{número \, de \, predicciones \, correctas}{número \, de \,predicciones \, totales}$$


- Precision: 

$$Precision = \dfrac{Verdaderos \, positivos}{Verdaderos \, positivos + Falsos \, positivos}$$

- Recall:

$$Recall = \dfrac{Verdaderos \, positivos}{Verdaderos \, positivos + Falsos \,negativos}$$

<p align="center">
  <img src="https://www.researchgate.net/publication/336402347/figure/fig3/AS:812472659349505@1570719985505/Calculation-of-Precision-Recall-and-Accuracy-in-the-confusion-matrix.ppm" width="350">
</p>

**To-Do:**

- [ ] Crear funciones para las métricas para clasificación binaria: Accuracy, Precision y Recall. 
- [ ] Compruebe el correcto funcionamiento de sus métricas a través de los test.

**Hint:** Utilice confusion_matrix para obtener TP, FP, FN y TN.



In [146]:
from sklearn.metrics import confusion_matrix

def accuracy(y_true, y_pred):
  ##### Código aquí #######
  cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
  return (cm[0, 0] + cm[1, 1]) / np.sum(cm)

def precision(y_true, y_pred):
  ##### Código aquí #######
  cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
  return cm[0, 0] / (cm[0, 0] + cm[0, 1])

def recall(y_true, y_pred):
  ##### Código aquí #######
  cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
  return cm[0, 0] / (cm[0, 0] + cm[1, 0])

In [147]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

y_true = [0, 1, 0, 0, 1, 0]
y_pred = [0, 0, 1, 0, 0, 1]

# Tests de metricas
assert accuracy_score(y_true, y_pred) == accuracy(y_true, y_pred)
assert recall_score(y_true, y_pred) == recall(y_true, y_pred)
assert precision_score(y_true, y_pred) == precision(y_true, y_pred)

### 1.1.4 Entrenemos los pipelines

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="300">
</p>


Entrene en base al pipeline generado en los pasos anteriores, procure separar los datos de entrenamiento en un conjunto de entrenamiento y de prueba (la proporción queda a su juicio). Una vez entrenado, muestre los resultados para cada uno de los clasificadores y comente lo que observa de los resultados obtenidos usando las métricas creadas anteriormente, ¿podrían mejorarse los resultados?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar el pipeline.
- [ ] Utilizar las métricas codificadas anteriormente y analizar los resultados.


In [114]:
### Código aquí ###
X = df_players.drop(columns=['Name', 'Nationality', 'Club_Position', 'National_Position'])
y = df_players['National_Position']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

pipeline1.fit(X_train, y_train)
pipeline2.fit(X_train, y_train)
pipeline3.fit(X_train, y_train)

y_test1 = pipeline1.predict(X_test)
y_test2 = pipeline2.predict(X_test)
y_test3 = pipeline3.predict(X_test)

print("Arbol de decisión\nAcc: {:.3f}\tRec: {:.3f}\tPres: {:.3f}\n\n".format(accuracy(y_test, y_test1), recall(y_test, y_test1), precision(y_test, y_test1)))
print("Random Forest\nAcc: {:.3f}\tRec: {:.3f}\tPres: {:.3f}\n\n".format(accuracy(y_test, y_test2), recall(y_test, y_test2), precision(y_test, y_test2)))
print("K nearest neightboor\nAcc: {:.3f}\tRec: {:.3f}\tPres: {:.3f}\n\n".format(accuracy(y_test, y_test3), recall(y_test, y_test3), precision(y_test, y_test3)))


Arbol de decisión
Acc: 0.903	Rec: 0.223	Pres: 0.238


Random Forest
Acc: 0.943	Rec: 0.750	Pres: 0.112


K nearest neightboor
Acc: 0.938	Rec: 0.423	Pres: 0.041




**Respuesta**:

```
En general dieron mejores resultados con Random Forest, aunque una baja precisión para todos, y los peores resultados con Knn.

Los resultados se podrían mejorar encontrando mejores hiperparámetros con GridSearch, también utilizando validación cruzada, obteniendo más datos, 
variando el porcentaje de separación de conjuntos train/test, utilizando otros modelos con mayor capacidad, entre otras.
```

## 1.2 Predicción de posiciones de jugadores. (2 puntos)

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="300">
</p>

### 1.2.1 Creación de nuevas labels.



No conforme con el trabajo de la sección 1.1, Renacin les *ordena* que construyan un clasificador nuevo, pero para predecir la posición de los jugadores del dataset. Para lograr esto, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, con esto les recalca que no deben utilizar los valores ```Sub``` y ```Res``` de este atributo, pidiendo que agrupen los valores en los siguientes grupos:

```python
Ataque=['ST', 'CAM','RS','LF','LAM','LS','RF','RAM','CF']
MedioCampistas=['RCM','LW', 'RW', 'LCM','LM', 'RM', 'CM']
Defensas=['LCB','RB','RCB', 'LDM','LB','CDM','RDM','CB','RWB','LWB']
Arquero =['GK']
```

Con lo señalado, Renacin les señala que no deberian utilizar el label utilizado en la sección 1.1, ya que el problema de clasificación ahora es de posiciones.

**To-Do**:

- [ ] Aplique las etiquetas de ```Ataque```, ```MedioCampista```, ```Defensa``` y ```Arquero``` en los valores señalados en esta sección.
- [ ] Muestre como se distribuyen los datos etiquetados.

In [148]:
df_players = df_players[df_players['Club_Position'] != 'Sub']
df_players = df_players[df_players['Club_Position'] != 'Res']
df_players = df_players.dropna(axis=0)

df_players.loc[df_players['Club_Position'].isin(['ST', 'CAM','RS','LF','LAM','LS','RF','RAM','CF']), 'Club_Position'] = 'Ataque'
df_players.loc[df_players['Club_Position'].isin(['RCM','LW', 'RW', 'LCM','LM', 'RM', 'CM']), 'Club_Position'] = 'MedioCampistas'
df_players.loc[df_players['Club_Position'].isin(['LCB','RB','RCB', 'LDM','LB','CDM','RDM','CB','RWB','LWB']), 'Club_Position'] = 'Defensas'
df_players.loc[df_players['Club_Position'].isin(['GK']), 'Club_Position'] = 'Arquero'

In [149]:
print(f"Cantidad de datos etiquetados como delantero: {(df_players.Club_Position == 'Ataque').sum()}")
print(f"Cantidad de datos etiquetados como defensa: {(df_players.Club_Position == 'Defensas').sum()}")
print(f"Cantidad de datos etiquetados como medio campistas: {(df_players.Club_Position == 'MedioCampistas').sum()}")
print(f"Cantidad de datos etiquetados como arqueros: {(df_players.Club_Position == 'Arquero').sum()}")

Cantidad de datos etiquetados como delantero: 1252
Cantidad de datos etiquetados como defensa: 3185
Cantidad de datos etiquetados como medio campistas: 1880
Cantidad de datos etiquetados como arqueros: 632


### 1.2.2 Entrenar un pipeline

Aplicando el mismo preprocesamiento solicitado en 1.1.2, genere un pipeline con un clasificador a gusto y evalúe al modelo de clasificación utilizando la **librería `scikit-learn`** (esto debido a que este es un problema multi clase y las funciones solicitadas anteriormente era para una clasificación binaria).

De los resultados obtenidos, ¿El modelo pudo clasificar mejor este problema o el de la sección 1.1?, Comente sus resultados.

**To-Do**:

- [ ] Genere un pipeline similar al visto en la sección 1.1.2.
- [ ] Aplique las métricas ofrecidas por la librería scikit-learn.
- [ ] Responda la pregunta del enunciado.

In [160]:
### Código aquí ###
enc = OneHotEncoder(handle_unknown='ignore')

pipeline = Pipeline(
    [
        ("ct", ct),
        ("svm", RandomForestClassifier())
    ]
)

X = df_players.drop(columns=['Name', 'Nationality', 'Club_Position'])
y = df_players['Club_Position']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print("Random Forest\nAcc: {:.3f}\tRec: {:.3f}\tPres: {:.3f}\n\n".format(accuracy(y_test, y_pred), recall(y_test, y_pred), precision(y_test, y_pred)))

  (0, 3)	1.0
  (1, 3)	1.0
  (2, 3)	1.0
  (3, 1)	1.0
  (4, 0)	1.0
  (5, 0)	1.0
  (6, 1)	1.0
  (7, 3)	1.0
  (8, 1)	1.0
  (9, 0)	1.0
  (10, 3)	1.0
  (11, 3)	1.0
  (12, 1)	1.0
  (13, 1)	1.0
  (14, 2)	1.0
  (15, 2)	1.0
  (16, 1)	1.0
  (17, 3)	1.0
  (18, 1)	1.0
  (19, 3)	1.0
  (20, 3)	1.0
  (21, 1)	1.0
  (22, 3)	1.0
  (23, 2)	1.0
  (24, 2)	1.0
  :	:
  (6924, 2)	1.0
  (6925, 2)	1.0
  (6926, 2)	1.0
  (6927, 2)	1.0
  (6928, 2)	1.0
  (6929, 2)	1.0
  (6930, 3)	1.0
  (6931, 3)	1.0
  (6932, 2)	1.0
  (6933, 3)	1.0
  (6934, 3)	1.0
  (6935, 3)	1.0
  (6936, 2)	1.0
  (6937, 0)	1.0
  (6938, 2)	1.0
  (6939, 3)	1.0
  (6940, 3)	1.0
  (6941, 3)	1.0
  (6942, 2)	1.0
  (6943, 2)	1.0
  (6944, 3)	1.0
  (6945, 2)	1.0
  (6946, 2)	1.0
  (6947, 3)	1.0
  (6948, 2)	1.0


TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]


**Respuesta**:

```
Comente de resultados observados.
```

## 1.3 Clustering Sobre el Dataset (2 puntos)

<p align="center">
  <img src="https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F1832816%2F349289c622faaff3afc517a46fcf4c71%2Fk-means.jpg?generation=1577258176400530&alt=media" width="200">
</p>

### 1.3.1 Elección de numero de clusters

<p align="center">
  <img src="https://media1.tenor.com/images/a85e9df2f5784d6513dce853ba9d5755/tenor.gif?itemid=18229271" width="200">
</p>





Entrene un k-means con 3, 4 y 5 clusters. ¿Las agrupaciones son lógicas usando un análisis visual?, Grafique los clusters con el código dispuesto más abajo y comente cual de las opciones presenta una mejor agrupación.

En base al análisis realizado, ubique la posición de los cluster en la cancha. Solo señale a que posición equivaldría tomando de referencia:

- Ataque
- Medio campistas
- Defensas
- Arquero 

**To-Do**:

- [ ] Escoger un numero de cluster y justificar elección.
- [ ] Comentar si las agrupaciones son lógicas a través de un análisis visual.
- [ ] Graficar los resultados obtenidos y visualizar las agrupaciones obtenidas.


In [ ]:
from umap import UMAP

def graficar_clusters(clusters, preprocessed_data = preprocessed_data,posicion = df_players.Club_Position.values):
  # Entreguele a esta función solo los clusters, el resto de parametros debería funcionar por default.
  umap = UMAP()
  projections = umap.fit_transform(preprocessed_data)
  fig = px.scatter(x=projections[:,0], y=projections[:,1], color=clusters, hover_data={'posicion': posicion}) 
  fig.show()

In [ ]:
### Código aquí ###

**Respuesta**:

```
Comente de resultados observados.
```

### 1.3.2 Búsqueda Número Óptimo de Clusters: Método del Codo **(Opcional 0.5 puntos)**

<p align="center">
  <img src="https://c.tenor.com/fk4p5HzTi5gAAAAC/elbow-high-five.gif" width="300">
</p>





Renancin muy indeciso de sus resultados le comenta que desea realizar un último trabajo. En esta última tarea le solicita la agrupación de los datos obtenidos en la sección 1.2 utilizando K-means. Para esto usted deberá visualizar a través del método del codo el número óptimo de agrupaciones para los datos. 

**Hint:** Revisar las clases y encontrar el código.

**To-Do**:

- [ ] Pre-procesar los datos con el mismo preprocesamiento aplicado en 1.2
- [ ] Utilizar el método del codo para obtener el número de clusters.

In [ ]:
### Código aquí ###

**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>